<a href="https://colab.research.google.com/github/Asritha-Kandula/FMML_M1L1.ipynb/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [16]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [17]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [18]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [19]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [20]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [21]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [22]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [23]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [24]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [25]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [26]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [27]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [28]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [29]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [30]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Averaging the validation accuracy across multiple splits is a technique used to assess the performance of a machine learning model more reliably. This approach helps to mitigate the effects of outliers or variations in the dataset, which can sometimes skew results from a single data split.

Here’s how it works:

1. Cross-validation: In k-fold cross-validation, the dataset is split into k subsets (folds). The model is trained k times, each time using a different fold as the validation set and the remaining k-1 folds for training. The performance metric (e.g., accuracy) is calculated for each fold, and the final performance estimate is the average of these k results.


2. Why it's consistent: The average accuracy across multiple splits smooths out the variance introduced by different random train-test splits. Some splits might have an easier or harder validation set due to randomness, but averaging multiple splits helps to provide a more generalizable estimate of the model's ability to perform on unseen data.


3. Variance reduction: With a single train-test split, the results can be influenced by how the data is partitioned. If the validation set is particularly easy or hard, the performance metric may not reflect the model’s true generalization ability. Multiple splits give a broader view of the model’s performance, reducing such biases.


4. More reliable results: By averaging over multiple splits, you get a more robust estimate of your model’s expected performance on unseen data. This is especially important when the dataset is small or when you want to ensure that the results are not overly influenced by a particular data split.



In summary, using multiple splits and averaging validation accuracy helps produce a more consistent and dependable estimate of model performance.










2. Does it give more accurate estimate of test accuracy?
Averaging validation accuracy across multiple splits does not directly give a more accurate estimate of test accuracy; however, it provides a better estimate of how the model is likely to perform on unseen data.

Here’s the distinction:

1. Validation Accuracy: The goal of averaging validation accuracy across multiple splits is to estimate how well your model is expected to generalize. It simulates how the model might perform on unseen data, but it is still based on the data that was used for training (just partitioned differently across the splits).


2. Test Accuracy: Test accuracy refers to how well the model performs on an entirely separate test set that was not used during training or validation. The test set provides a more accurate measure of how the model will perform in a real-world scenario, on data it has never seen before.


3. Relation to Generalization: Averaging validation accuracy improves the estimate of generalization performance by reducing the bias introduced by a single random split. But, the true test accuracy is unknown until you evaluate the model on the actual test set after final training. So, while averaging validation accuracy gives a more reliable estimate of expected performance, it does not directly equate to the true test accuracy.



In summary, while averaging validation accuracy gives a more reliable indication of how the model will generalize to unseen data, it is still not the same as the final test accuracy, which can only be determined after testing on a separate dataset.




3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations, or more specifically the number of cross-validation folds or splits, has a significant impact on the reliability of the estimate of model performance. Increasing the number of iterations generally leads to a more stable and accurate estimate of the model’s generalization ability, but it comes with diminishing returns and computational cost.

1. Impact of More Iterations:
When you increase the number of iterations (e.g., more folds in cross-validation), the model is evaluated on more unique subsets of the data. This provides a better approximation of how the model will perform on unseen data. The greater the number of iterations, the smaller the variance in your performance estimate, as each split contributes less bias and error to the final average. The law of large numbers suggests that as the number of iterations increases, the estimate converges closer to the true performance of the model.


2. Diminishing Returns:
While increasing the number of folds or iterations improves the estimate, the benefits become diminishing after a certain point. For example, in k-fold cross-validation, the model undergoes k iterations, and as you increase k, the variance in performance estimates decreases. However, beyond a certain threshold (often around 10-fold), the gains in accuracy of the estimate start to plateau. After this point, the additional computational cost may outweigh the improvement in estimate quality.


3. Bias-Variance Trade-off:
With fewer iterations, there is a higher variance in the estimate because each fold or iteration is based on a smaller subset of the data. This leads to a higher likelihood of the estimate being skewed by specific characteristics of those subsets. With more iterations, the model is exposed to a larger and more diverse range of data points, which helps mitigate this variance. However, if the dataset is large enough, a reasonable number of iterations (e.g., 5-10) often provides a good balance between computational efficiency and result accuracy.


4. Computational Cost:
It’s important to acknowledge that increasing the number of iterations increases the computational burden. Each additional iteration requires retraining the model and evaluating it on different data partitions. For very large datasets or complex models, the cost of performing many iterations may become prohibitive, so a careful balance between the number of iterations and computational resources is necessary.


5. Generalization Error Estimation:
More iterations lead to a more robust estimate of the model’s expected generalization error. By ensuring the model is tested on various data splits, the resulting estimate becomes more reflective of real-world performance. This is especially critical when working with smaller datasets, where a higher number of iterations allows each data point to be included in the validation set multiple times, leading to more reliable performance measures.



In summary, while increasing the number of iterations (or folds) improves the stability and accuracy of your model’s performance estimate, the improvements tend to diminish after a certain point. The optimal number of iterations strikes a balance between reducing variance in the estimate and managing computational efficiency. Generally, a higher number of iterations yields a more trustworthy estimate, especially in settings where generalization is critical, but the trade-off in terms of computational resources must be considered.




4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Yes, increasing the number of iterations (e.g., using more folds in cross-validation) can help deal with very small training or validation datasets, but there are important trade-offs and limitations to consider.

How Increasing Iterations Helps with Small Datasets:

1. Maximizing Data Utilization:
With more iterations (e.g., higher k in k-fold cross-validation), a larger proportion of the data is used for training in each iteration. For example, in 10-fold cross-validation, 90% of the data is used for training in each iteration, leaving only 10% for validation. This allows you to make the most of a small dataset by repeatedly using all data points for both training and validation across iterations.


2. Reducing Overfitting to Specific Splits:
With a small dataset, random splits can lead to high variability in performance estimates because some splits may have "easy" or "hard" validation sets. Increasing iterations (e.g., leave-one-out cross-validation, where each data point is a validation set once) reduces the likelihood that any single split disproportionately impacts the overall estimate, providing a more robust evaluation.


3. Better Generalization Estimates:
For small datasets, averaging performance across many splits gives a more reliable estimate of generalization. Since all data points participate in both training and validation, you avoid biases that may arise from excluding certain data points in a single validation set.




---

Limitations and Challenges:

1. Small Validation Sets:
Even with more iterations, each validation set in small datasets will have very few samples. This can lead to noisy or unreliable validation metrics, especially if the dataset is highly imbalanced or contains outliers.


2. Overfitting Risks:
When using very small datasets, the model may overfit the training data even with cross-validation. More iterations won't fundamentally solve the problem of limited data diversity or representativeness.


3. Computational Cost:
Increasing iterations, especially with small datasets, can be computationally expensive since the model must be trained repeatedly. For very small datasets, leave-one-out cross-validation (n iterations for n samples) can become impractical for complex models.




---

Strategies to Supplement Small Data:

1. Data Augmentation:
If possible, increase the effective dataset size using techniques like synthetic data generation or augmentation (e.g., transformations, oversampling).


2. Simpler Models:
Use simpler models (e.g., linear models, regularization) that are less likely to overfit small datasets.


3. Transfer Learning:
Leverage pre-trained models or representations if applicable, especially for tasks like image classification or natural language processing.


4. Bootstrapping:
Consider bootstrapping methods (resampling with replacement) to complement cross-validation and further estimate variability in model performance.




---

Conclusion:

Increasing iterations can help extract more information from small datasets and provide better generalization estimates. However, the fundamental limitation of small datasets—lack of diversity and representativeness—cannot be fully addressed by more iterations alone. Complementary techniques like augmentation, transfer learning, or collecting more data should be considered when possible.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.